In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [19]:
# Read the csv file 
amzn_df = pd.read_csv(Path("Resources/amzn.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
amzn_df ['time'] = pd.to_datetime(
    amzn_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
amzn_df['time'] = amzn_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
amzn_df.set_index('time', inplace=True)

# Review df
amzn_df.head()          

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,Lower Band #3,Plot,Plot.1,Plot.2,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,,,,
2012-03-14 09:30:00-04:00,9.1825,9.215995,9.057000,9.112999,9.128665,9.128665,9.128665,9.128665,9.128665,9.128665,9.128665,9.215995,9.057000,9.1825,73994680,107206355.0,9.597910,9.606610,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-15 09:30:00-04:00,9.1010,9.221500,9.015000,9.221500,9.152667,9.152667,9.152667,9.152667,9.152667,9.152667,9.152667,9.221500,9.015000,9.1010,83212340,103564913.0,9.594164,9.602464,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-16 09:30:00-04:00,9.1640,9.284000,9.117501,9.252500,9.218000,9.218000,9.218000,9.218000,9.218000,9.218000,9.218000,9.284000,9.117501,9.1640,98696480,95934268.0,9.590765,9.598431,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-19 09:30:00-04:00,9.1725,9.334000,9.150000,9.276000,9.253333,9.253333,9.253333,9.253333,9.253333,9.253333,9.253333,9.334000,9.150000,9.1725,78080380,92419724.0,9.587633,9.594651,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-20 09:30:00-04:00,9.2440,9.720500,9.144000,9.616500,9.493667,9.493667,9.493667,9.493667,9.493667,9.493667,9.493667,9.720500,9.144000,9.2440,183346180,94828072.0,9.587920,9.591678,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Filter column and only keep ones needed
amzn_df = amzn_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
amzn_df = amzn_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
amzn_df = amzn_df.dropna()

# Review the DataFrame
amzn_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2012-03-14 09:30:00-04:00,9.1825,9.215995,9.057000,9.112999,9.128665,73994680,107206355.0,9.597910
2012-03-15 09:30:00-04:00,9.1010,9.221500,9.015000,9.221500,9.152667,83212340,103564913.0,9.594164
2012-03-16 09:30:00-04:00,9.1640,9.284000,9.117501,9.252500,9.218000,98696480,95934268.0,9.590765
2012-03-19 09:30:00-04:00,9.1725,9.334000,9.150000,9.276000,9.253333,78080380,92419724.0,9.587633
2012-03-20 09:30:00-04:00,9.2440,9.720500,9.144000,9.616500,9.493667,183346180,94828072.0,9.587920


In [4]:
# Read the csv file 
meta_df = pd.read_csv(Path("Resources/meta.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
meta_df ['time'] = pd.to_datetime(
    meta_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
meta_df['time'] = meta_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
meta_df.set_index('time', inplace=True)

# Review df
meta_df.head()

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-05-18 09:30:00-04:00,42.05,45.00000,38.00,38.2318,40.410600,40.410600,40.410600,40.410600,40.410600,40.410600,...,580587776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-21 09:30:00-04:00,36.53,36.66000,33.00,34.0300,34.563333,34.563333,34.563333,34.563333,34.563333,34.563333,...,168309808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-22 09:30:00-04:00,32.61,33.59000,30.94,31.0000,31.843333,31.843333,31.843333,31.843333,31.843333,31.843333,...,102053808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-23 09:30:00-04:00,31.37,32.50000,31.36,32.0000,31.953333,31.953333,31.953333,31.953333,31.953333,31.953333,...,73721120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-24 09:30:00-04:00,32.95,33.21001,31.77,33.0300,32.670003,32.670003,32.670003,32.670003,32.670003,32.670003,...,50275872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Filter column and only keep ones needed
meta_df = meta_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
meta_df = meta_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
meta_df = meta_df.dropna()

# Review the DataFrame
meta_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2013-03-07 09:30:00-05:00,27.57001,28.67500,27.47000,28.57800,28.241000,74540128,46413816.0,25.669904
2013-03-08 09:30:00-05:00,28.42500,28.47000,27.73000,27.96001,28.053337,44198832,46896756.8,25.692691
2013-03-11 09:30:00-04:00,28.00999,28.64000,27.83000,28.14000,28.203333,35642064,46793423.2,25.717042
2013-03-12 09:30:00-04:00,28.10001,28.32001,27.60001,27.83000,27.916673,27569584,46303815.2,25.738067
2013-03-13 09:30:00-04:00,27.62000,27.64999,26.92000,27.08000,27.216663,39619440,43610005.6,25.751419


In [6]:
# Read the csv file 
tsla_df = pd.read_csv(Path("./Resources/tsla.csv"))

# Convert 'time' column from timestamp (seconds since epoch) to acutal time
tsla_df ['time'] = pd.to_datetime(
    tsla_df['time'],
    unit = 's',
    infer_datetime_format=True,
    utc=True
)    
# Convert timezone from UTC to Eastern Time
tsla_df['time'] = tsla_df['time'].dt.tz_convert('US/Eastern')

# Set 'time' column as the index 
tsla_df.set_index('time', inplace=True)
                      
# Review df
tsla_df.head()   

,open,high,low,close,VWAP,Upper Band #1,Lower Band #1,Upper Band #2,Lower Band #2,Upper Band #3,...,Volume,Volume MA,EMA,Smoothing Line,Developing Poc,Developing VA High,Developing VA Low,Developing Poc.1,Developing VA High.1,Developing VA Low.1
time,,,,,,,,,,,,,,,,,,,,,
2012-03-12 09:30:00-04:00,2.312664,2.419330,2.306665,2.400664,2.375553,2.375553,2.375553,2.375553,2.375553,2.375553,...,29467829,16848705.00,1.929784,1.921918,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-13 09:30:00-04:00,2.433997,2.439331,2.366664,2.405998,2.403998,2.403998,2.403998,2.403998,2.403998,2.403998,...,15024660,16731543.65,1.934522,1.925750,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-14 09:30:00-04:00,2.399998,2.399998,2.319998,2.352664,2.357553,2.357553,2.357553,2.357553,2.357553,2.357553,...,12771972,16011617.15,1.938683,1.929832,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-15 09:30:00-04:00,2.351998,2.365331,2.318664,2.333331,2.339109,2.339109,2.339109,2.339109,2.339109,2.339109,...,8573948,14366955.50,1.942610,1.934130,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-16 09:30:00-04:00,2.326664,2.392664,2.321998,2.354665,2.356442,2.356442,2.356442,2.356442,2.356442,2.356442,...,10938550,13249154.35,1.946710,1.938462,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Filter column and only keep ones needed
tsla_df = tsla_df[['open','high','low','close','VWAP','Volume','Volume MA','EMA']]

#Rename EMA to 200EMA
tsla_df = tsla_df.rename(columns={'EMA':'200EMA'})

#Dropping NAN Values 
tsla_df = tsla_df.dropna()

# Review the DataFrame
tsla_df.head()

,open,high,low,close,VWAP,Volume,Volume MA,200EMA
time,,,,,,,,
2012-03-12 09:30:00-04:00,2.312664,2.419330,2.306665,2.400664,2.375553,29467829,16848705.00,1.929784
2012-03-13 09:30:00-04:00,2.433997,2.439331,2.366664,2.405998,2.403998,15024660,16731543.65,1.934522
2012-03-14 09:30:00-04:00,2.399998,2.399998,2.319998,2.352664,2.357553,12771972,16011617.15,1.938683
2012-03-15 09:30:00-04:00,2.351998,2.365331,2.318664,2.333331,2.339109,8573948,14366955.50,1.942610
2012-03-16 09:30:00-04:00,2.326664,2.392664,2.321998,2.354665,2.356442,10938550,13249154.35,1.946710


### Trading Algo (Signal Generation)

In [8]:
# Import the finta library
from finta import TA

#set some options for display and troubleshooting
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

In [13]:
# Define periods for fast and slow EMAs (triggers)
fast_ema = 9
slow_ema = 40

# Initialize list of ticker dfs for use in for loop
ticker_df = [amzn_df,meta_df,tsla_df]

# For each ticker dataframe in the list:
for df in ticker_df:
    
    # Initialize empty Signal column with 0.0 default value
    df['Signal'] = 0.0
    
    # Calculate Fast EMA 
    df['Fast_EMA'] = TA.EMA(df,fast_ema)
    
    # Calculate Slow EMA
    df['Slow_EMA'] = TA.EMA(df,slow_ema)
    
    # Generate Signal value ("long if fast_ema > slow_ema", otherwise flip short)
    df['Signal'] = np.where(
        df['Slow_EMA'] < df['Fast_EMA'], 1.0, -1.0)
    
    # Create Entry/Exit column and fill with the differences (trades) from Signal column
    df['Entry/Exit'] = df['Signal'].diff()
    
    # Calculate % returns
    df['pct_returns'] = df['close'].pct_change()
    
    #Drop any NA values
    df.dropna()


C:\Users\giffo\AppData\Local\Temp\ipykernel_21220\3102969288.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Signal'] = 0.0
C:\Users\giffo\AppData\Local\Temp\ipykernel_21220\3102969288.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fast_EMA'] = TA.EMA(df,fast_ema)
C:\Users\giffo\AppData\Local\Temp\ipykernel_21220\3102969288.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [15]:
# Filter each dataframe to 10yr period
tsla_df = tsla_df.loc['05-2012':'05-2022']
meta_df = meta_df.loc['05-2012':'05-2022']
amzn_df = amzn_df.loc['05-2012':'05-2022']

In [16]:
tsla_df

,open,high,low,close,VWAP,Volume,Volume MA,200EMA,Signal,Fast_EMA,Slow_EMA,Entry/Exit,pct_returns
time,,,,,,,,,,,,,
2012-05-01 09:30:00-04:00,2.208664,2.280665,2.208664,2.251998,2.247109,9884214,16794143.75,2.036327,-1.0,2.251998,2.251998,NaN,NaN
2012-05-02 09:30:00-04:00,2.233331,2.292664,2.225998,2.262664,2.260442,7458772,16338969.05,2.038579,1.0,2.257924,2.257464,2.0,0.004736
2012-05-03 09:30:00-04:00,2.260664,2.266664,2.141998,2.163999,2.190887,12631617,13608434.55,2.039827,-1.0,2.219430,2.224739,-2.0,-0.043606
2012-05-04 09:30:00-04:00,2.154665,2.163999,2.093331,2.121998,2.126443,18712308,13411940.35,2.040645,-1.0,2.186425,2.197096,0.0,-0.019409
2012-05-07 09:30:00-04:00,2.130665,2.171998,2.107331,2.164665,2.147998,17369552,13038563.00,2.041879,-1.0,2.179952,2.189945,0.0,0.020107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-24 09:30:00-04:00,217.843115,217.973115,206.856460,209.386457,211.405344,89092604,92445460.00,291.611393,-1.0,235.560272,281.478938,0.0,-0.069255
2022-05-25 09:30:00-04:00,207.949792,223.106444,207.669826,219.599780,216.792017,92139416,93204607.20,290.894860,-1.0,232.368174,278.460442,0.0,0.048777
2022-05-26 09:30:00-04:00,220.473113,239.556394,217.886449,235.909764,231.117536,106003450,92257347.15,290.347744,-1.0,233.076492,276.384800,0.0,0.074271
